<a href="https://colab.research.google.com/github/rakshith3101/IG_gateway_task/blob/main/M1_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/anger-ratings-0to1.train.txt http://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/fear-ratings-0to1.train.txt http://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/joy-ratings-0to1.train.txt http://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/sadness-ratings-0to1.train.txt
!wget http://saifmohammad.com/WebDocs/EmoInt%20Test%20Data/anger-ratings-0to1.test.target.txt http://saifmohammad.com/WebDocs/EmoInt%20Test%20Data/fear-ratings-0to1.test.target.txt http://saifmohammad.com/WebDocs/EmoInt%20Test%20Data/joy-ratings-0to1.test.target.txt  http://saifmohammad.com/WebDocs/EmoInt%20Test%20Data/sadness-ratings-0to1.test.target.txt

--2024-03-10 06:31:05--  http://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/anger-ratings-0to1.train.txt
Resolving saifmohammad.com (saifmohammad.com)... 192.185.17.122
Connecting to saifmohammad.com (saifmohammad.com)|192.185.17.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95686 (93K) [text/plain]
Saving to: ‘anger-ratings-0to1.train.txt’

anger-ratings-0to1. 100%[===================>]  93.44K  --.-KB/s    in 0.04s   

2024-03-10 06:31:06 (2.60 MB/s) - ‘anger-ratings-0to1.train.txt’ saved [95686/95686]

--2024-03-10 06:31:06--  http://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/fear-ratings-0to1.train.txt
Reusing existing connection to saifmohammad.com:80.
HTTP request sent, awaiting response... 200 OK
Length: 133186 (130K) [text/plain]
Saving to: ‘fear-ratings-0to1.train.txt’

fear-ratings-0to1.t 100%[===================>] 130.06K  --.-KB/s    in 0.01s   

2024-03-10 06:31:06 (11.1 MB/s) - ‘fear-ratings-0to1.train.txt’ saved [133186/133186]

--20

In [ ]:
#train_data
import io
file_content=['/content/anger-ratings-0to1.train.txt','/content/fear-ratings-0to1.train.txt','/content/joy-ratings-0to1.train.txt','/content/sadness-ratings-0to1.train.txt']
output_file='/content/training_data.txt'
with open(output_file, 'w') as output:
    # Write the header line to the output file
    output.write("id\ttweet\tlabel\tintensity\n")

    # Loop through each input file
    for file_name in file_content:
        # Open the input file in read mode
        with open(file_name, 'r') as input_file:
            # Skip the header line in the input file
            next(input_file)

            # Copy the content of the input file to the output file
            output.write(input_file.read())

print(f"Combined data from {', '.join(file_content)} into {output_file}.")

Combined data from /content/anger-ratings-0to1.train.txt, /content/fear-ratings-0to1.train.txt, /content/joy-ratings-0to1.train.txt, /content/sadness-ratings-0to1.train.txt into /content/training_data.txt.


In [ ]:
#test_data
import io
file_content=['/content/anger-ratings-0to1.test.target.txt','/content/fear-ratings-0to1.test.target.txt','/content/joy-ratings-0to1.test.target.txt','/content/sadness-ratings-0to1.test.target.txt']
output_file='/content/test_data.txt'
with open(output_file, 'w') as output:
    # Write the header line to the output file
    output.write("id\ttweet\tlabel\tintensity\n")

    # Loop through each input file
    for file_name in file_content:
        # Open the input file in read mode
        with open(file_name, 'r') as input_file:
            # Skip the header line in the input file
            next(input_file)

            # Copy the content of the input file to the output file
            output.write(input_file.read())

print(f"Combined data from {', '.join(file_content)} into {output_file}.")

Combined data from /content/anger-ratings-0to1.test.target.txt, /content/fear-ratings-0to1.test.target.txt, /content/joy-ratings-0to1.test.target.txt, /content/sadness-ratings-0to1.test.target.txt into /content/test_data.txt.


In [ ]:
import pandas as pd
train_data = pd.read_csv('/content/training_data.txt', delimiter='\t')


In [ ]:
train_data.head()

,id,tweet,label,intensity
0,10001,So my Indian Uber driver just called someone t...,anger,0.896
1,10002,@DPD_UK I asked for my parcel to be delivered ...,anger,0.896
2,10003,so ef whichever butt wipe pulled the fire alar...,anger,0.896
3,10004,Don't join @BTCare they put the phone down on ...,anger,0.896
4,10005,My blood is boiling,anger,0.875


In [ ]:
label_mapping={'anger':0,'fear':1,'joy':2,'sadness':3}
train_data['label']=train_data['label'].map(label_mapping)
train_data.head()
train_data.info()

In [ ]:
test_data=pd.read_csv('/content/test_data.txt',delimiter='\t')
test_data['label']=test_data['label'].map(label_mapping)
test_data.head()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3138 entries, 0 to 3137
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         3138 non-null   int64 
 1   tweet      3138 non-null   object
 2   label      3138 non-null   int64 
 3   intensity  3138 non-null   object
dtypes: int64(2), object(2)
memory usage: 98.2+ KB


In [ ]:
test_data=test_data.drop(columns=['intensity'])

In [ ]:
!pip install neattext
import neattext.functions as nfx
train_data['clean_text']= train_data['tweet'].apply(nfx.remove_userhandles)
train_data['clean_text']=train_data['clean_text'].apply(nfx.remove_hashtags)
train_data['clean_text']=train_data['clean_text'].apply(nfx.remove_stopwords)
train_data['clean_text']=train_data['clean_text'].apply(nfx.remove_emojis)
train_data['clean_text']=train_data['clean_text'].apply(nfx.remove_punctuations)
train_data['clean_text'] = train_data['clean_text'].apply(lambda x: x.lower())
train_data.head()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 2.8 MB/s eta 0:00:00


,id,tweet,label,intensity,clean_text
0,10001,So my Indian Uber driver just called someone t...,0,0.896,indian uber driver called n word moving vehicl...
1,10002,@DPD_UK I asked for my parcel to be delivered ...,0,0.896,asked parcel delivered pick store address
2,10003,so ef whichever butt wipe pulled the fire alar...,0,0.896,ef whichever butt wipe pulled fire alarm davis...
3,10004,Don't join @BTCare they put the phone down on ...,0,0.896,join phone you talk rude taking money acc will...
4,10005,My blood is boiling,0,0.875,blood boiling


In [ ]:
test_data['clean_text']=test_data['tweet'].apply(nfx.remove_userhandles)
test_data['clean_text']=test_data['clean_text'].apply(nfx.remove_hashtags)
test_data['clean_text']=test_data['clean_text'].apply(nfx.remove_stopwords)
test_data['clean_text']=test_data['clean_text'].apply(nfx.remove_emojis)
test_data['clean_text']=test_data['clean_text'].apply(nfx.remove_punctuations)
test_data['clean_text']=test_data['clean_text'].apply(lambda x: x.lower())


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_data['encoded_labels'] = label_encoder.fit_transform(train_data['label'])
test_data['encoded_labels']=label_encoder.fit_transform(test_data['label'])

In [ ]:
x_train=train_data.clean_text
y_train=train_data.label
x_test=test_data.clean_text
y_test=test_data.label

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer()
vect.fit(x_train)

CountVectorizer()

In [ ]:
x_train_dtm = vect.transform(x_train)
x_test_dtm = vect.transform(x_test)

In [ ]:
vect_tunned = CountVectorizer(stop_words='english', ngram_range=(1,2), min_df=0.1, max_df=0.7, max_features=100)
vect_tunned

CountVectorizer(max_df=0.7, max_features=100, min_df=0.1, ngram_range=(1, 2),
                stop_words='english')

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel='linear', random_state =32)
model.fit(x_train_dtm, y_train)
#predicting output for test data
pred = model.predict(x_test_dtm)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

0.607074569789675